# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


# Main blocks of code:
* Dataset import
* Data Exploration & Cleaning
* Merging datasets
* Saving cleaned and merged dataset

#### Questions to be answered:
* Which age groups complete the offer?
* How well can we predict the success of offers?

### Importing libraries and datasets

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import datetime
from time import time

# read in the json files
portfolio = pd.read_json(r'C:\Users\BernadettKepenyes\Documents\GitHub\starbucks-offer-success\portfolio.json', orient='records', lines=True)
profile = pd.read_json(r'C:\Users\BernadettKepenyes\Documents\GitHub\starbucks-offer-success\profile.json', orient='records', lines=True)
transcript = pd.read_json(r'C:\Users\BernadettKepenyes\Documents\GitHub\starbucks-offer-success\transcript.json', orient='records', lines=True)

### Data Exploration & Cleaning

#### Portfolio dataset

In [2]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
portfolio.shape

(10, 6)

In [4]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [5]:
portfolio['id'].nunique()

10

In [6]:
# encode channels 
mlb = MultiLabelBinarizer()
channel = pd.DataFrame(mlb.fit_transform(portfolio['channels']), columns=mlb.classes_, index=portfolio.index)

# drop channels column with new channel columns
portfolio = portfolio.drop(['channels'], axis=1)

# concatenate the original 'po' df with new channel df
portfolio = pd.concat([portfolio,channel], join='inner', axis=1)
portfolio

,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,0,1


In [7]:
# encode offer types 
offer_type = pd.get_dummies(portfolio['offer_type'])

# concatenate offer_type and portfolio
portfolio = pd.concat([portfolio, offer_type], join='inner', axis=1)

# drop offer_type column
portfolio = portfolio.drop(['offer_type'], axis=1)

portfolio

,reward,difficulty,duration,id,email,mobile,social,web,bogo,discount,informational
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,1,0
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,1,0
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0,0,0,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,0,0
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,1,1,0,1,0,1,0


In [8]:
# change duration days column to duration hours
portfolio['duration'] = portfolio['duration'] * 24
portfolio

,reward,difficulty,duration,id,email,mobile,social,web,bogo,discount,informational
0,10,10,168,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,120,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,96,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1
3,5,5,168,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0
4,5,20,240,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0
5,3,7,168,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,1,0
6,2,10,240,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,1,0
7,0,0,72,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0,0,0,1
8,5,5,120,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,0,0
9,2,10,168,2906b810c7d4411798c6938adc9daaa5,1,1,0,1,0,1,0


In [9]:
# rename columns
portfolio.rename(columns={'reward':'meta_reward','difficulty':'meta_required_spending', 'duration':'meta_max_hours','id': 'offer_id', 'email': 'meta_email', 'mobile':'meta_mobile', 'social':'meta_social', 'web':'meta_web', 'bogo':'meta_bogo', 'discount':'meta_discount', 'informational':'meta_informational'}, inplace=True)
portfolio.head(2)

,meta_reward,meta_required_spending,meta_max_hours,offer_id,meta_email,meta_mobile,meta_social,meta_web,meta_bogo,meta_discount,meta_informational
0,10,10,168,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,120,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0


#### Profile dataset

In [10]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [11]:
profile.shape

(17000, 5)

In [12]:
# convert became_member_on to date
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile.head(10)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,2017-09-25,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,2017-10-02,NaN
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0
9,None,118,8974fc5686fe429db53ddde067b88302,2016-11-22,NaN


In [13]:
# add total days of membership, as the date of today - date of became member

def create_days(pf):
    '''
    Input: it takes the current date and extract the date of became a member
    Output: number of days of membership
    '''
    current_date = pd.to_datetime('today')
    
    return(current_date - pf['became_member_on'])

profile['profile_membership_days'] = profile.apply(create_days, axis=1)
profile['profile_membership_days'] = profile['profile_membership_days'].dt.days
profile.drop(['became_member_on'], axis=1, inplace=True)

profile.head()

,gender,age,id,income,profile_membership_days
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,NaN,1516
1,F,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1363
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,NaN,1001
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1430
4,None,118,a03223e636434f42ac4c3df47e8bac43,NaN,1343


In [14]:
profile['gender'].nunique()

3

In [15]:
# encode gender column
profile['gender'] = profile['gender'].apply(lambda x: 1 if x =='M' else 2 if x == 'F' else 0)
profile.head(10)

,gender,age,id,income,profile_membership_days
0,0,118,68be06ca386d4c31939f3a4f0e3dd783,NaN,1516
1,2,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1363
2,0,118,38fe809add3b4fcf9315a9694bb96ff5,NaN,1001
3,2,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1430
4,0,118,a03223e636434f42ac4c3df47e8bac43,NaN,1343
5,1,68,e2127556f4f64592b11af22de27a7932,70000.0,1078
6,0,118,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,1291
7,0,118,68617ca6246f4fbc85e91a2a49552598,NaN,1284
8,1,65,389bc3fa690240e798340f5a15918d5c,53000.0,1154
9,0,118,8974fc5686fe429db53ddde067b88302,NaN,1598


In [16]:
profile['age'].value_counts()

118    2175
58      408
53      372
51      363
54      359
       ... 
100      12
96        8
98        5
99        5
101       5
Name: age, Length: 85, dtype: int64

In [17]:
profile[profile.age == 118][['gender', 'age', 'income']]

,gender,age,income
0,0,118,NaN
2,0,118,NaN
4,0,118,NaN
6,0,118,NaN
7,0,118,NaN
...,...,...,...
16980,0,118,NaN
16982,0,118,NaN
16989,0,118,NaN
16991,0,118,NaN


In [18]:
# 118 represents missing values
profile['age'] = profile['age'].apply(lambda x: np.nan if x == 118 else x)
# drop these missing values
profile.dropna(inplace=True)
profile.head(20)

,gender,age,id,income,profile_membership_days
1,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363
3,2,75.0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1430
5,1,68.0,e2127556f4f64592b11af22de27a7932,70000.0,1078
8,1,65.0,389bc3fa690240e798340f5a15918d5c,53000.0,1154
12,1,58.0,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,1244
13,2,61.0,aa4862eba776480b8bb9c68455b8c2e1,57000.0,1305
14,1,26.0,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,2611
15,2,62.0,31dda685af34476cad5bc968bdb01c53,71000.0,1883
16,1,49.0,62cf5e10845442329191fc246e7bcea3,52000.0,2338
18,1,57.0,6445de3b47274c759400cd68131d91b4,42000.0,1194


In [19]:
profile.shape

(14825, 5)

In [20]:
profile.duplicated().sum()

0

In [21]:
# rename id column to member_id
profile.rename(columns={'id': 'member_id','gender':'profile_gender', 'age':'profile_age', 'income':'profile_income'}, inplace=True)
profile.head(2)

,profile_gender,profile_age,member_id,profile_income,profile_membership_days
1,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363
3,2,75.0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1430


#### Transcript dataset

In [22]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [23]:
transcript.shape

(306534, 4)

In [24]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [25]:
transcript.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [26]:
# create dummies variable from event column
transcript = pd.get_dummies(transcript , columns=['event'] , prefix='' ,prefix_sep='')
transcript.head()

,person,value,time,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0


In [27]:
# extract values from value column 
transcript = pd.concat([transcript, transcript['value'].apply(pd.Series)], axis=1)
transcript.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer id,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [28]:
# clean duplicated columns
transcript['offer_id_new']=np.where(transcript['offer id'].isnull() & transcript['offer_id'].notnull(),transcript['offer_id'],transcript['offer id'])

# drop unnecessary offer_id columns
transcript.drop(['offer id','offer_id'],axis=1,inplace=True)

# rename columns
transcript.rename(columns={'person': 'member_id','time':'transcript_time', 'offer_id_new':'offer_id', 'amount':'transcript_amount', 'reward':'transcript_reward', 'offer completed':'offer_completed', 'offer received':'offer_received', 'offer viewed': 'offer_viewed', 'transaction': 'offer_transaction'},inplace=True)

# drop value column
transcript.drop(['value'], axis=1, inplace=True)

transcript.head()

,member_id,transcript_time,offer_completed,offer_received,offer_viewed,offer_transaction,transcript_amount,transcript_reward,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,NaN,NaN,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0


In [29]:
# offer groups
transcript.offer_id.value_counts()

fafdcd668e3743c1bb461111dcafc2a4    20241
2298d6c36e964ae4a3e7e9706d1fb8c2    20139
f19421c1d4aa40978ebb69ca19b0e20d    19131
4d5c57ea9a6940dd891ad53e9dbe8da0    18222
ae264e3637204a6fb9bb56bc8210ddfd    18062
9b98b8c7a33c4b65b9aebfe6a799e6d9    16202
2906b810c7d4411798c6938adc9daaa5    15767
5a8bc65990b245e5a138643cd4eb9837    14305
0b1e1539f2cc45b7b9fa7c272da2e1d7    13751
3f207df678b143eea3cee63160fa8bed    11761
Name: offer_id, dtype: int64

### Merging datasets

In [30]:
df_member_id = pd.merge(profile, transcript, on='member_id')
df = pd.merge(df_member_id, portfolio, how="left", on='offer_id')
df.head(3)

,profile_gender,profile_age,member_id,profile_income,profile_membership_days,transcript_time,offer_completed,offer_received,offer_viewed,offer_transaction,...,meta_reward,meta_required_spending,meta_max_hours,meta_email,meta_mobile,meta_social,meta_web,meta_bogo,meta_discount,meta_informational
0,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,18,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,144,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,408,0,1,0,0,...,5.0,5.0,168.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


In [31]:
# change offer_id to integers
df['offer_id'] = df['offer_id'].map({'fafdcd668e3743c1bb461111dcafc2a4':1, '2298d6c36e964ae4a3e7e9706d1fb8c2':2, 'f19421c1d4aa40978ebb69ca19b0e20d':3, '4d5c57ea9a6940dd891ad53e9dbe8da0':4, 'ae264e3637204a6fb9bb56bc8210ddfd':5, '9b98b8c7a33c4b65b9aebfe6a799e6d9':6, '2906b810c7d4411798c6938adc9daaa5':7, '5a8bc65990b245e5a138643cd4eb9837':8, '0b1e1539f2cc45b7b9fa7c272da2e1d7':9, '3f207df678b143eea3cee63160fa8bed':10})
df.head()

,profile_gender,profile_age,member_id,profile_income,profile_membership_days,transcript_time,offer_completed,offer_received,offer_viewed,offer_transaction,...,meta_reward,meta_required_spending,meta_max_hours,meta_email,meta_mobile,meta_social,meta_web,meta_bogo,meta_discount,meta_informational
0,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,18,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,144,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,408,0,1,0,0,...,5.0,5.0,168.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,504,0,1,0,0,...,0.0,0.0,96.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,528,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# fill nan with 0
df = df.fillna(0)
df.head()

,profile_gender,profile_age,member_id,profile_income,profile_membership_days,transcript_time,offer_completed,offer_received,offer_viewed,offer_transaction,...,meta_reward,meta_required_spending,meta_max_hours,meta_email,meta_mobile,meta_social,meta_web,meta_bogo,meta_discount,meta_informational
0,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,18,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,144,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,408,0,1,0,0,...,5.0,5.0,168.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,504,0,1,0,0,...,0.0,0.0,96.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,528,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# drop channels - offer_id already determines the type of offer
df.drop(['meta_email', 'meta_mobile', 'meta_social', 'meta_web', 'meta_bogo', 'meta_discount', 'meta_informational'], axis=1, inplace=True)
df.head()

,profile_gender,profile_age,member_id,profile_income,profile_membership_days,transcript_time,offer_completed,offer_received,offer_viewed,offer_transaction,transcript_amount,transcript_reward,offer_id,meta_reward,meta_required_spending,meta_max_hours
0,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,18,0,0,0,1,21.51,0.0,0.0,0.0,0.0,0.0
1,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,144,0,0,0,1,32.28,0.0,0.0,0.0,0.0,0.0
2,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,408,0,1,0,0,0.00,0.0,6.0,5.0,5.0,168.0
3,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,504,0,1,0,0,0.00,0.0,10.0,0.0,0.0,96.0
4,2,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,1363,528,0,0,0,1,23.22,0.0,0.0,0.0,0.0,0.0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272762 entries, 0 to 272761
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   profile_gender           272762 non-null  int64  
 1   profile_age              272762 non-null  float64
 2   member_id                272762 non-null  object 
 3   profile_income           272762 non-null  float64
 4   profile_membership_days  272762 non-null  int64  
 5   transcript_time          272762 non-null  int64  
 6   offer_completed          272762 non-null  uint8  
 7   offer_received           272762 non-null  uint8  
 8   offer_viewed             272762 non-null  uint8  
 9   offer_transaction        272762 non-null  uint8  
 10  transcript_amount        272762 non-null  float64
 11  transcript_reward        272762 non-null  float64
 12  offer_id                 272762 non-null  float64
 13  meta_reward              272762 non-null  float64
 14  meta

In [35]:
# save cleaned and merged data
df.to_csv(r'C:\Users\BernadettKepenyes\Documents\GitHub\starbucks-offer-success\clean_df.csv', index=False)